We create a supplementary table that is an excel file with the following sheets:
- target - annotated for updating to moalmanac
- moalmanac - aneuploidy
- moalmanac - copy_number
- moalmanac - germline variant
- moalmanac - knockdown
- moalmanac - microsatellite stability
- moalmanac - mutational burden
- moalmanac - mutational signature
- moalmanac - neoantigen burden
- moalmanac - rearrangement
- moalmanac - silencing
- moalmanac - somatic variant

In [1]:
import openpyxl
import pandas as pd
import glob 

filename = 'supplementary-table-2'
folder = f'../../../tables/supplementary-table-02'
!mkdir -p $folder
outname = f'{folder}/{filename}.xlsx'

options = {}
options['strings_to_formulas'] = False
options['strings_to_urls'] = False

writer = pd.ExcelWriter(outname)

target = pd.read_excel('../target/almanac-comparison.xlsx')
target.to_excel(writer, sheet_name='TARGET')

sheet_names = []
handles = glob.glob('../moalmanac-db/content/*')
for handle in sorted(handles):
    dtype = handle.split('/')[-1].split('.tsv')[0]
    if dtype == 'microsatellite_stability':
        dtype = 'msi'
    sheet_name = f'moalmanac-{dtype}'
    sheet_names.append(sheet_name)
    
    tmp = pd.read_csv(handle, sep='\t')
    tmp.to_excel(writer, sheet_name=sheet_name, index=False)
    
writer.save()